# Puzzles

## "Easy as ABC"
Source: https://twitter.com/Five_Triangles/status/1015738379454046208

Rules:
Letters A,B,C must each appear exactly once in every row and column.
Outside letters must be closest to that letter in the respective row or column, e.g., C must be highest in column 1.

![The puzzle](https://pbs.twimg.com/media/DhigAqEXkAAsGtN.jpg)

## My solution

In [1]:
from copy import deepcopy

In [2]:
def checkrow(row, req):
    '''Make sure a given row isn't breaking the two given requirements.
    (This function can also be used to check columns.)'''

    # The row may contain up to 1 A, 1 B, 1 C, and the rest can be spaces
    n = len(row)
    if row.count('A') > 1 or row.count('B') > 1 or row.count('C') > 1 or row.count(' ') > n-3:
        return False
    
    # If the row has any letters, check that the first one matches the requirement
    rowstr = ''.join(row)
    if len(rowstr.strip()) > 0 and req[0] and rowstr.strip()[0] != req[0]:
        return False
    
    # If the row is full, check that the last letter matches the requirement
    if len(rowstr) == len(row) and req[1] and rowstr.strip()[-1] != req[1]:
        return False
    
    return True

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def checkgrid(grid, row_reqs, col_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g, r) for g,r in zip(grid, row_reqs)):
        return False
    if not all(checkrow(g, r) for g,r in zip(cols(grid), col_reqs)):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 2) + '\n')
          .join([' | '.join(g) for g in grid]))  

# In each spot in a grid, letters will be tested in this order
cycle = {'':' ', ' ':'A', 'A':'B', 'B':'C'}
    
def solve_it(row_reqs, col_reqs):
    '''For an ABC grid problem specified by certain row and 
    column requirements, find and display all solutions.'''

    solutions = []
    
    # Size of the problem
    n = len(row_reqs)

    # List of grids (the first grid is just all blank)
    grids = [[[''] * n for _ in range(n)]]

    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:

        # At the current position in the grid, try the next letter
        try:
            grids[-1][curry][currx] = cycle[grids[-1][curry][currx]]

        # If there are no more letters to try, back up to the previous position
        except KeyError: 
            grids.pop()
            currx -= 1
            if currx < 0:
                currx  = n-1
                curry -= 1
            continue

        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1) and checkgrid(grids[-1], row_reqs, col_reqs):
            solutions.append(deepcopy(grids[-1]))
            
        # Otherwise, if we're not currently breaking any rules, proceed to the next position
        elif checkrow(grids[-1][curry], row_reqs[curry]) and checkrow(col(grids[-1], currx), col_reqs[currx]):
            grids.append(deepcopy(grids[-1]))
            currx += 1
            if currx > n-1:
                currx  = 0
                curry += 1
    
    return solutions

In [3]:
soln = solve_it(
    row_reqs = [(None, None),
                ('A', None),
                (None, 'A'),
                (None, 'B'),
                (None, 'B'),
                (None, None)],
    col_reqs = [('C', None),
                ('C', 'A'),
                ('A', None),
                (None, None),
                (None, 'B'),
                (None, 'A')]
)

for s in soln: 
    printgrid(s)
    print('\n\n')

  | C |   |   | A | B
----------------------
  |   | A | B |   | C
----------------------
  | B |   |   | C | A
----------------------
C | A | B |   |   |  
----------------------
A |   |   | C | B |  
----------------------
B |   | C | A |   |  





## "1 to 9"
Source: https://twitter.com/1to9puzzle/status/1018120012966621184

![The puzzle](https://pbs.twimg.com/media/DiEWj8gXkAIXFeE.jpg)

## Mysolution

In [4]:
from itertools import product, permutations

In [5]:
# Find the correct labels for all the sums we might possibly encounter
sum_dict = {}
max_size = 9+8+7+6

# Evens and odds
for k in range(max_size//2+1): sum_dict[2*k] = 'E'
for k in range(max_size//2+1): sum_dict[2*k+1] = 'O'

# Primes (bleh!)
primes = list(range(2,max_size+1))
for j in range(2,int(max_size**.5)+1):
    for i in range(2,max_size//j+1):
        try:
            primes.remove(i*j)
        except:
            pass
for k in primes: sum_dict[k] = 'P'
    
# Square, cube, triangular numbers
for k in range(int(max_size**.5)+1): sum_dict[k**2] = 'S'
for k in range(int(max_size**.3334)+1): sum_dict[k**3] = 'C'
for k in range(1,int((max_size*2+1)**.5)+1): sum_dict[k*(k+1)//2] = 'T'

In [6]:
def checkgrid(grid):
    '''Determine whether a given grid obeys the rules.'''
    for i,j in product(range(7), range(7)):
        if (type(grid[i][j]) == str 
            and grid[i][j] != sum_dict[
                grid[i+1][j] + grid[i-1][j] 
                + grid[i][j+1] + grid[i][j-1]
            ]):
            return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 2) + '\n')
          .join([' | '.join(str(x) for x in g) for g in grid]))  

In [7]:
# Set up the grid for the puzzle
grid = [[0] * 7 for _ in range(7)]

# Label the dark squares
grid[1][4] = 'T'
grid[2][1] = 'C'
grid[2][3] = 'T'
grid[3][2] = 'E'
grid[3][4] = 'E'
grid[4][3] = 'E'
grid[4][5] = 'E'
grid[5][2] = 'P'

# Prepare to handle the light squares
lights = [(i,j) for i,j in product(range(1,6), range(1,6)) 
          if (i+j) % 2 == 0
          and (i,j) not in [(1,5),(5,1),(1,1),(5,5),(3,3)]]
grid[3][3] = 7

# Try each possible way to fill the light squares and show the solution(s)
for digits in permutations([1,2,3,4,5,6,8,9]):
    for (i,j),d in zip(lights, digits):
        grid[i][j] = d
    if checkgrid(grid):
        printgrid(grid)
        print('\n\n')

0 | 0 | 0 | 0 | 0 | 0 | 0
--------------------------
0 | 0 | 0 | 2 | T | 0 | 0
--------------------------
0 | C | 5 | T | 1 | 0 | 0
--------------------------
0 | 3 | E | 7 | E | 8 | 0
--------------------------
0 | 0 | 9 | E | 6 | E | 0
--------------------------
0 | 0 | P | 4 | 0 | 0 | 0
--------------------------
0 | 0 | 0 | 0 | 0 | 0 | 0



